Modeling Notebook:

# Loading Packages

In [6]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Loading Data

In [7]:
# Define the path to your CSV file
file_path = r"C:/Users/johne/Downloads/IWC_Work_Orders_Extract.csv"
# file_path = r"C:/Users/matt/Downloads/IWC_Work_Orders_Extract.csv"
# Read the CSV file into a pandas DataFrame
master_df = pd.read_csv(file_path)




# GROUP 1: Matt, Alex, Carson
group_1_df = master_df.copy()

# GROUP 2: Nick S., Nick G.
group_2_df = master_df.copy()




#IMPORTANT
# REPLACE DF PLACEHOLDER
df_col_drop_lasso=group_2_df.drop(['ORDER_ID','PLANT_ID', 'FUNCTIONAL_AREA_NODE_1_MODIFIED', 
                                   'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'FUNCTIONAL_AREA_NODE_3_MODIFIED', 'FUNCTIONAL_AREA_NODE_4_MODIFIED',
                                   'FUNCTIONAL_AREA_NODE_5_MODIFIED', 'EQUIP_VALID_TO', 'EXECUTION_START_DATE', 
                                   'EXECUTION_FINISH_DATE', 'EQUIP_START_UP_DATE', 'EQUIP_VALID_FROM', 'EQUIP_VALID_TO', 'ACTUAL_START_TIME', 'ACTUAL_FINISH_TIME'],axis=1)
#how to handle nodes? Equip start/vaid date?

df_filtered_lasso = df_col_drop_lasso.dropna(subset=['ORDER_DESCRIPTION', 'MAINTENANCE_PLAN', 'MAINTENANCE_ITEM','MAINTENANCE_TYPE_DESCRIPTION', 'FUNCTIONAL_LOC', 'EQUIPMENT_ID'], how='all')

df_filtered_lasso.isnull().sum()/len(df_filtered_lasso)


C:\Users\johne\AppData\Local\Temp\ipykernel_11696\2456862225.py:5: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(file_path)


PRODUCTION_LOCATION             0.000000
ACTUAL_WORK_IN_MINUTES          0.000000
MAINTENANCE_PLAN                0.477051
MAINTENANCE_ITEM                0.477051
MAINTENANCE_ACTIVITY_TYPE       0.000000
ORDER_DESCRIPTION               0.000096
MAINTENANCE_TYPE_DESCRIPTION    0.000000
FUNCTIONAL_LOC                  0.000045
EQUIPMENT_ID                    0.000000
EQUIPMENT_DESC                  0.727730
EQUIP_CAT_DESC                  0.727730
dtype: float64

# Removing null values 

In [8]:
# Create a new DataFrame by dropping rows with NaN values
group_2_df_cleaned = df_filtered_lasso.dropna()

# Verify that there are no NaN values remaining
print("Number of NaN values in each column:\n", group_2_df_cleaned.isna().sum())

# Display the new shape of the DataFrame
print("Original shape:", group_2_df.shape)
print("Shape after dropping NaNs:", group_2_df_cleaned.shape)

Number of NaN values in each column:
 PRODUCTION_LOCATION             0
ACTUAL_WORK_IN_MINUTES          0
MAINTENANCE_PLAN                0
MAINTENANCE_ITEM                0
MAINTENANCE_ACTIVITY_TYPE       0
ORDER_DESCRIPTION               0
MAINTENANCE_TYPE_DESCRIPTION    0
FUNCTIONAL_LOC                  0
EQUIPMENT_ID                    0
EQUIPMENT_DESC                  0
EQUIP_CAT_DESC                  0
dtype: int64
Original shape: (1427264, 25)
Shape after dropping NaNs: (57067, 11)


# Glypse of Data

In [9]:
# Display the first 5 rows of the DataFrame
print("Head of the DataFrame:")
print(group_2_df_cleaned.head())

Head of the DataFrame:
   PRODUCTION_LOCATION  ACTUAL_WORK_IN_MINUTES MAINTENANCE_PLAN  \
5                MONZA                    18.0       G291VM1437   
71                COTA                    36.0     000000058177   
72                COTA                    60.0     000000058177   
73                COTA                    36.0     000000058177   
74                COTA                    36.0     000000058177   

    MAINTENANCE_ITEM MAINTENANCE_ACTIVITY_TYPE  \
5            40534.0                   Planned   
71          235113.0                   Planned   
72          235113.0                   Planned   
73          235113.0                   Planned   
74          235113.0                   Planned   

                           ORDER_DESCRIPTION  MAINTENANCE_TYPE_DESCRIPTION  \
5   TANKS AIR STORAGE FOR MECH WHILE SHUTDOW  Preventive Maintenance Order   
71     WEEKLY BOTTLE BURST TESTER INSPECTION  Preventive Maintenance Order   
72     WEEKLY BOTTLE BURST TESTER INSPE

Filter Further, by plant, silverstone

In [10]:
df_filtered_lasso_silverstone = group_2_df_cleaned[group_2_df_cleaned['PRODUCTION_LOCATION'] == 'SILVERSTONE']

In [11]:
df_filtered_lasso_silverstone.shape

(249, 11)

# Creating new variables:

# Model